In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import tensorflow as tf 
import matplotlib.pyplot as plt
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image # dealing with images 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
        
directory = '/kaggle/input/early-detection-of-3d-printing-issues'
df_train = pd.read_csv(directory + '/train.csv')
df_test = pd.read_csv(directory + '/test.csv')

df_train_0 = df_train[df_train['has_under_extrusion'] == 0].sample(25)
df_train_1 = df_train[df_train['has_under_extrusion'] == 1].sample(25)

img_path = '/kaggle/working/training_images/0'
os.makedirs(img_path)

for index in range(df_train_0['img_path'].count()):
    image = Image.open(directory + '/images/' + df_train_0['img_path'].iloc[index])
    image = image.save(img_path + "/" + str(index + 1) + '.jpg')
    
img_path = '/kaggle/working/training_images/1'
os.makedirs(img_path)

for index in range(df_train_1['img_path'].count()):
    image = Image.open(directory + '/images/' + df_train_1['img_path'].iloc[index])
    image = image.save(img_path + "/" + str(index + 1) + '.jpg')

train = ImageDataGenerator(rescale=1/255)
   

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
training_data = train.flow_from_directory("/kaggle/working/training_images",
                                         target_size = (255,255),
                                         batch_size = 5,
                                         class_mode = 'binary')

Found 50 images belonging to 2 classes.


In [4]:
training_data.class_indices

{'0': 0, '1': 1}

In [5]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (5,5), activation = "relu", input_shape = (255,255,3)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation = "relu"),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation = "relu"),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation = "relu"),
    tf.keras.layers.Dense(1, activation = "sigmoid")
])

In [6]:
model.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [7]:
model_fit = model.fit(
    training_data,
    steps_per_epoch = 5,
    epochs = 10
)

Epoch 1/10
5/5 [==============================] - 3s 261ms/step - loss: 1.3475 - accuracy: 0.3600
Epoch 2/10
5/5 [==============================] - 1s 255ms/step - loss: 0.7296 - accuracy: 0.5200
Epoch 3/10
5/5 [==============================] - 1s 250ms/step - loss: 0.7119 - accuracy: 0.5200
Epoch 4/10
5/5 [==============================] - 1s 252ms/step - loss: 0.7279 - accuracy: 0.3200
Epoch 5/10
5/5 [==============================] - 1s 252ms/step - loss: 0.6766 - accuracy: 0.6000
Epoch 6/10
5/5 [==============================] - 1s 249ms/step - loss: 0.6515 - accuracy: 0.7600
Epoch 7/10
5/5 [==============================] - 1s 243ms/step - loss: 0.5983 - accuracy: 0.7600
Epoch 8/10
5/5 [==============================] - 1s 243ms/step - loss: 0.5947 - accuracy: 0.7200
Epoch 9/10
5/5 [==============================] - 1s 246ms/step - loss: 0.4432 - accuracy: 0.9200
Epoch 10/10
5/5 [==============================] - 1s 239ms/step - loss: 0.3143 - accuracy: 0.8800


In [ ]:
hue_list = []

for index in range(df_test['img_path'].count()):
    image = Image.open(directory + '/images/' + df_test['img_path'].iloc[index])
    image = image.resize((255,255))
    img_arr = np.asarray(img_arr)
    img_arr = np.expand_dims(img_arr, axis = 0)
    value = model.predict(img_arr)
    hue_list.append(value[0][0])

1/1 [==============================] - 0s 34ms/step


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1/1 [==============================] - 0s 40ms/step


In [ ]:
hue_list1 = [round(x) for x in hue_list]

In [ ]:
df_test['has_under_extrusion'] = hue_list1
df_final = df_test[['img_path','has_under_extrusion']]
df_final.to_csv('/kaggle/working/test1.csv', index = False)